# Deepspeed 1, 2 & 3 benchmark
This model being trained has the same settings as raven 1B5 model.
- Layer count: 24
- Embed size: 2048

The goal is to validate the trainer across deepspeed 1, 2 & 3 - with and without offload. All other training params remain constant. And benchmarking them accordingly

## What does deepspeed 1, 2 & 3 do (With/Without CPU offload) ??

Instead of simply splitting the dataset being trained, and having a full copy of nearly everything in all GPU's (aka DDP / DeepSpeed 1).

Deepspeed 2, keeps a full copy of the model weights on each GPU, but splits the training gradient descent memory usage into multiple GPUs, or offload it into CPU memory (+ CPU offload option).

Deepspeed 3, takes it a step further, and distributes the model weights across all the GPUs, drastically lowering the vram requirement, while increasing the amount of GPU to GPU traffic drastically. Gradient descent memory is still split across multiple GPUs, with the option to offload into CPU memory (Same as deepspeed 2)

Finally, Deepspeed 3, also introduce options to further offload such model weights / gradient descent, more into CPU memory or NVMe. However this option was not enabled or explored in the following benchmarks.

See more here: https://huggingface.co/docs/transformers/main_classes/deepspeed

## Benchmark results

Benchmark was done on 20th Aug 2023. With Torch 2.0.1, Cuda 11.8. On 8x3090, via vast.ai
All benchmarks was done with ctx length of 4096

(@TODO - conslidate and update result)

---

| Deepspeed Strat       | Time (8 * 3090)  | % difference | VRAM Usage | RAM Usage (peak / mean) |
|-----------------------|------------------|--------------|------------|-------------------------|
| Stage 1               | 13 mins : 29 sec | -            | 65.11%     | 6.7 / 3.1 ~GB           |
| Stage 2               | 13 mins : 29 sec | -            | 55.64%     | 6.7 / 3.1 ~GB           |
| Stage 2 + CPU offload | 14 mins : 56 sec | 10.754 %     | 44.61%     | 9.4 / 8.0 ~GB           |
| Stage 3               | 14 mins : 11 sec | 5.191 %      | 61.32%     | 6.7 / 2.3 ~GB           |
| Stage 3 + CPU offload | 17 mins : 20 sec | 28.553 %     | 44.88%     | 7.4 / 7.2 ~GB           |

---

> ^ note in theory deepspeed 3 uses less vram then deepspeed 2, however it will also try to use up more ram then its needed for "cache" items if possible, maxing out to the same level as deepspeed 2 here
>
> Torch.JIT was enabled for deepspeed 2, But was disabled for deepspeed 3 (not compatible). Torch.compile was disabled


## Configure and apply your preferred settings

Adjust your desired deepspeed settings, and gpu device count.

Enable/Disable WANDB here as well ( Enabled by default, as we need the loss curve for this experiment )

( note you will need to rerun this cell, if you restart your env )

In [1]:
GPU_DEVICES="auto"
ENABLE_WANDB=False
WANDB_PREFIX="infctx-v5-deepspeed-test"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: False
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/rwkv-x-playground/notebook/trainer-v5-validation
TRAINER_DIR: /root/rwkv-x-playground/RWKV-v5
PROJECT_DIR: /root/rwkv-x-playground


In [2]:
# Init the model
!cd "{TRAINER_DIR}" && \
    python3 ./init_model.py \
        --n_layer 24 --n_embd 2048 \
        --vocab_size neox --skip-if-exists \
        "../model/L24-D2048-neox-v5base-init.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
---- Initializing model ----
No of layers: 24
Embedding size: 2048
Output model path: ../model/L24-D2048-neox-v5base-init.pth
Vocab size: 50277
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
50277 2048  -0.0001 emb.weight
2048  2048  1.0  blocks.0.att.receptance.weight
2048  2048  1.0  blocks.0.att.key.weight
2048  2048  1.0  blocks.0.att.value.weight
2048  2048  0    blocks.0.att.output.weight
8192  2048  1.0  blocks.0.ffn.key.weight
2048  2048  0    blocks.0.ffn.receptance.weight
2048  8192  0    blocks.0.ffn.value.weight
2048  2048  1.0  blocks.1.att.receptance.weight
2048  2048  1.0  blocks.1.att.key.weight
2048  2048  1.0  blocks.1.att.value.weight
2048  2048  0    blocks.1.att.output.weight
8192  2048  1.0  blocks.1.ffn.key.weight
2048  2048  0    blocks.1.ffn.receptance.weight
2048  8192  0    blocks.1.f

In [3]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/baseline-4096.yaml"

Found cached dataset parquet (/root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 156.13it/s]


# Deepspeed 1

In [4]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/baseline-4096.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (deepspeed_stage_1, train-ctx=4096, data-ctx=4096)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.devices="{GPU_DEVICES}"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/trainer-v5-validation/config/baseline-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-deepspeed-test (deepspeed_stage_1, train-ctx=4096, data-ctx=4096)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/trainer-v5-validation/config/baseline-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-deepspeed-test (deepspeed_stage_1, train-ctx=4096, data-ctx=4096)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto'].
  rank_zero_warn(
Global

# Deepspeed 2

In [5]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/baseline-4096.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (deepspeed_stage_2, train-ctx=4096, data-ctx=4096)" \
        --trainer.strategy="deepspeed_stage_2" \
        --trainer.devices="{GPU_DEVICES}"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/trainer-v5-validation/config/baseline-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-deepspeed-test (deepspeed_stage_2, train-ctx=4096, data-ctx=4096)', '--trainer.strategy=deepspeed_stage_2', '--trainer.devices=auto'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/trainer-v5-validation/config/baseline-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-deepspeed-test (deepspeed_stage_2, train-ctx=4096, data-ctx=4096)', '--trainer.strategy=deepspeed_stage_2', '--trainer.devices=auto'].
  rank_zero_warn(
Global

# Deepspeed 2 + Offload
Perform a full 1 epoch training run of training context size = 1024. With deepspeed 2

In [6]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/baseline-4096.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (deepspeed_stage_2_offload, train-ctx=4096, data-ctx=4096)" \
        --trainer.strategy="deepspeed_stage_2_offload" \
        --trainer.devices="{GPU_DEVICES}"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/trainer-v5-validation/config/baseline-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-deepspeed-test (deepspeed_stage_2_offload, train-ctx=4096, data-ctx=4096)', '--trainer.strategy=deepspeed_stage_2_offload', '--trainer.devices=auto'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/trainer-v5-validation/config/baseline-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-deepspeed-test (deepspeed_stage_2_offload, train-ctx=4096, data-ctx=4096)', '--trainer.strategy=deepspeed_stage_2_offload', '--trainer.devices=

# Deepspeed 3
Perform a full 1 epoch training run of training context size = 1024. With deepspeed 3

In [7]:
!cd "{TRAINER_DIR}" && \
    export RWKV_JIT_ON=0 && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/baseline-4096.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (deepspeed_stage_3, train-ctx=4096, data-ctx=4096)" \
        --trainer.strategy="deepspeed_stage_3" \
        --trainer.devices="{GPU_DEVICES}"

[RWKV.lightning_trainer.py] Detected deepspeed_stage_3, disabling JIT using RWKV_JIT_ON=0
Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-native' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/trainer-v5-validation/config/baseline-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-deepspeed-test (deepspeed_stage_3, train-ctx=4096, data-ctx=4096)', '--trainer.strategy=deepspeed_stage_3', '--trainer.devices=auto'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/trainer-v5-validation/config/baseline-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-deepspeed-test (deepspeed_stage_3, train-ctx=4096, data-ctx=4096)'

# Deepspeed 3 + offload
Perform a full 1 epoch training run of training context size = 1024. With deepspeed 3 + offload

In [8]:
!cd "{TRAINER_DIR}" && \
    export RWKV_JIT_ON=0 && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/baseline-4096.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (deepspeed_stage_3_offload, train-ctx=4096, data-ctx=4096)" \
        --trainer.strategy="deepspeed_stage_3_offload" \
        --trainer.devices="{GPU_DEVICES}"

[RWKV.lightning_trainer.py] Detected deepspeed_stage_3_offload, disabling JIT using RWKV_JIT_ON=0
Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-native' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/trainer-v5-validation/config/baseline-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-deepspeed-test (deepspeed_stage_3_offload, train-ctx=4096, data-ctx=4096)', '--trainer.strategy=deepspeed_stage_3_offload', '--trainer.devices=auto'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/trainer-v5-validation/config/baseline-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-deepspeed-test (deepspeed_stage_3_offload,